# LLM Powered Q/A System

Intelligent document assistant that can pull relevant answers from the text

> I chose the ACKO insurance document from the provided datasets

## TASK: 1
Design a prompt
Few shot prompting (5 samples)

In [18]:
prompt = """## SYSTEM:
You are a helpful assistant that can help users by answering questions and providing useful information. You will be given context required to answer the question. The user will ask you a question and you will answer it based on the context provided. If the context does not provide enough information to answer the question, you will say "I don't know". You will not make up any information or provide any opinions. You will only provide information that is present in the context.
## SAMPLES:
### Example 1:
user: Explain the insurance policy related to loss of pay because of hospitalization
asssistant: If an Insured Person suffers an Involuntary Unemployment due to an Illness or Injury, as specified in the Schedule / Certificate of Insurance, suffered or contracted during the Coverage Period, resulting in
loss of Income, then We will pay the monthly amount specified in the Policy Schedule / Certificate of Insurance against this Benefit, for the duration of such Unemployment, up to the number of months /
days specified in the Policy Schedule / Certificate of Insurance from the date of such Involuntary Unemployment.

### Example 2:
user: What insurace policies are present related to child education
assistant: We will pay the amount specified in the Policy Schedule / Certificate of Insurance at the frequency specified in the Policy Schedule / Certificate of Insurance in respect of each surviving Dependent Child,
irrespective of whether the child is an Insured Person under this Policy 

### Example 3:
user: I met with an accident and have to return to my place of residence, what are my options
assistant: If an Insured Person suffers an Injury due to an Accident that occurs during the Coverage Period and that Injury solely and directly requires the Insured Person to return to his place of residence, then We
will reimburse the amount incurred on tickets on a Common Carrier for the Insured Person’s travel back to his place of residence with addition or modification necessitated in the Common Carrier due to such
Illness/Injury and provided to the Insured Person, up to the limit specified in the Policy Schedule / Certificate of Insurance. This Benefit will be payable provided thathe Medical Practitioner treating the Insured Person certifies in
writing that the Insured Person is suffering from the Injury in respect of which the claim is being made.

### Example 4:
user: Unexpectedly my trip got delayed. Will I get a refund.
assistant: We will pay the amount specified in the Policy Schedule / Certificate of Insurance, if an Insured Person’s journey on a Common Carrier is delayed beyond the number of hours specified in the Policy Schedule
/ Certificate of Insurance of its scheduled departure or scheduled arrival time, during the Travel Period.

### Example 5:
user: I am denied accomodation at a hotel. Is there a policy that helps me on this?
assistant: We will reimburse the cost upto the limit specified in the Policy Schedule / Certificate of Insurance of alternative accommodation required by the Insured Person due to any cancellation of the Insured
Person’s booked and confirmed accommodation by a hotel or any other provider of accommodation.

## CONTEXT: 
{context}
## USER:
{question}
## ASSISTANT:
"""

## Build a basic Q/A system

- load the document
- create a retriever
- llm to answer

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
pdf_file_path = "../data/Acko-Group-Health-Insurance.pdf"


In [12]:
loader = PyPDFLoader(pdf_file_path)
docs = loader.load()
print(docs[10])  # checking random document
print(docs[0].metadata)

page_content='Acko General Insurance Limited 
Acko Group Health Insurance Policy 
 
 
 
Acko General Insurance Limited  
3rd Floor, F-wing, Lotus Corporate Park, Goregaon East, Mumbai, Maharashtra 400063 
 IRDAI Reg No: 157 | CIN: U66000MH2016PLC287385 | UIN: ACKHLGP20011V011920 
 www.acko.com | Toll free: 1860 266 2256 | Mail: hello@acko.com Page 11 of 95 
with the Primary Insured’s Policy and no Cumulative Bonus will be carried  forward to the split 
policies.  
d. Reduction in Sum Insured : If the Sum Insured has been reduced at the time of Renewal, the 
applicable Cumulative Bonus shall be calculated on the revised Sum Insured on pro -rata basis. 
d. Increase in Sum Insured : If the Sum Insured under the Policy has been increased at the time of 
Renewal the Cumulative Bonus shall be calculated on the Sum Insured of the last completed Policy 
Year. 
e. In case a claim is made in the expiring Coverage Period, which is notified after the acceptance of 
Renewal premium, the awarded Cum

In [14]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
# docs[10]

In [15]:
split_docs[10]

Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2019-11-28T16:35:19+05:30', 'moddate': '2019-11-28T16:35:19+05:30', 'source': '../data/Acko-Group-Health-Insurance.pdf', 'total_pages': 95, 'page': 1, 'page_label': '2'}, page_content='coverage category will be admissible subject to the fulfilment of the following conditions with respect to \nthe Insured Person’s Hospitalization: \ni. The Hospitalization of the Insured Person is caused solely and directly due to an Illness \ncontracted or an Injury sustained by the Insured Person , during the Coverage Period,  as \nspecified in the Policy Schedule / Certificate of Insurance. \nii. The Date of Admission is within the Coverage Period.')

In [16]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_documents(split_docs)

['2cd0192f-1ad2-4ce2-b0e5-97acf4cb3df5',
 'b2cbec38-6b51-423a-8825-83a2c41be107',
 'e26f9fa5-e324-4554-88f2-d06fc4064aff',
 '5161b4d6-1ddf-4279-8956-d89ca30584d7',
 'aa3b8fa1-756a-4f04-9e84-4e30ec0f501b',
 '35e85acc-1926-4a14-8c8d-0cb46fa88b07',
 'a99b95b9-06b9-4a9b-91cd-e86f5cb4b53a',
 '51137d94-dfe0-4b51-8185-413882cf35f1',
 '6a911847-6903-447e-9d95-3a2574f78cca',
 'a39be29f-57d4-4ccf-88f7-6500b91338b6',
 '430de3ba-d235-41bb-a755-4097eebd448c',
 '34d12207-c49e-475b-90e2-0c807c9532c7',
 '59b3f3cc-611c-4240-bb40-eab75950a36e',
 '3e08abdd-9815-4380-a712-7689956619d2',
 '5e7f669f-2ca9-49d7-a43e-7fc5431bcb15',
 '0c6ed50d-35fd-4b70-9f33-205165e5966e',
 'cdae5ec2-0eeb-4fca-9474-c4250f8871b2',
 'a59dc915-6508-48db-8a7d-8bc9c8100f50',
 '33fbf711-3ac4-46ee-93e7-77854c63ab7f',
 '075a770b-1392-4b27-8e50-689ebacb96ba',
 'a927143b-1aac-4e91-9766-ac92ab4908fc',
 'a5b86b4d-18ac-4fde-9723-f12450c4e4ba',
 '9a5d46ed-981e-4b8c-9484-7cfab9e5ac6a',
 'a13550df-54ca-444e-9672-e1ad9dedaf15',
 'db6d2e23-0f5a-

In [17]:
test_vdb = vector_store.similarity_search("child education", k=3)
for res in test_vdb:
    print(f"* {res.page_content} [{res.metadata}]")

* 2.2.1.5 Child Education Cover 
We will pay the amount specified in the Policy Schedule / Certificate of Insurance at the frequency 
specified in the Policy Schedule / Certificate of Insurance in respect of each surviving Dependent Child, 
irrespective of whether the child is an Insured Person under this Policy.  
For the purpose of this Benefit: [{'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2019-11-28T16:35:19+05:30', 'moddate': '2019-11-28T16:35:19+05:30', 'source': '../data/Acko-Group-Health-Insurance.pdf', 'total_pages': 95, 'page': 18, 'page_label': '19'}]
* 2.2.1.2 (Permanent  Total Disability), Benefit 2.2.1.3 (P ermanent Partial Disability) and Benefit 
2.2.1.4 (Temporary Total Disability)  in respect of the Insured Person will cumulatively exceed the 
Common Death or Disability Sum Insured then Our maximum, total and cumulative liability under 
any and all such claims will be limited to the Common Death or Disability Sum Insured. 
 

In [19]:
from langchain_core.prompts import PromptTemplate

prompt_temp = PromptTemplate.from_template(prompt)

In [26]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [27]:
# from langchain_community.llms import CTransformers

# llm = CTransformers(
#     model=r"D:\NK_Programming\ML\Akaike\Backend\Model\tinyllama-1.1b-chat-v1.0.Q5_K_M.gguf",
#     model_type="llama"
# )

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.1,
    max_retries=3
)

print("LLM Test")
print(llm.invoke("Hello there!"))

LLM Test
content="Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 38, 'total_tokens': 63, 'completion_time': 0.033333333, 'prompt_time': 0.004005868, 'queue_time': 0.23147563300000001, 'total_time': 0.037339201}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'stop', 'logprobs': None} id='run--1af03e96-cfd7-405d-b60d-40f3ae912647-0' usage_metadata={'input_tokens': 38, 'output_tokens': 25, 'total_tokens': 63}


In [22]:
def retriever(question):
    context_docs = vector_store.similarity_search(question, k=3)
    context = "\n-----\n".join([f"{doc.page_content}" for doc in context_docs])
    return context

In [23]:
def get_answer(question):
    context = retriever(question)
    prmpt = prompt_temp.invoke({"context":context, "question":question})
    return llm.invoke(prmpt)

In [29]:
answer = get_answer("what policies are present related to dental issues")
print(answer.content)

We will indemnify for any of the following Medical Expenses specified in Policy Schedule / Certificate of Insurance incurred during the Coverage Period, by the Insured:

- Amalgam filling 
- Composite/Resin filling 
- Root canal Treatment 
- Osseous Surgery 
- Periodontal scaling and root planning 
- Adjustments 
- Recement bridge 
- Routine extractions 
- Surgical removal of impacted tooth 
- Local or general Anaesthesia including sedation 

However, We will not be liable to make any payment in respect of Orthodontic Treatment, routine Treatment and dental implants.


In [30]:
answer2 = get_answer("explain any cashless option you provide")
print(answer2.content)

If an Insured Person requires medical treatment, We provide a Cashless Facility from a Network Provider. This means that the Insured Person can receive treatment without paying any amount upfront, and We will settle the bill directly with the Network Provider. 

Before availing a Cashless facility, the Policyholder / Insured Person can check the updated list to ensure that the Network Provider can provide Cashless Facility in respect of the Treatment required. 

However, We reserve the right to modify, add or restrict any Network Provider for providing Cashless facilities under the Policy. In some instances, We may deny Cashless Facility for Hospitalization due to insufficient Sum Insured or insufficient information to determine admissibility. In such cases, the Insured Person may be required to pay for the Treatment and submit the claim for reimbursement to Us, which will be considered subject to the Policy terms and conditions.
